In [1]:
!pip install anndata

You should consider upgrading via the '/home/roko/.cache/pypoetry/virtualenvs/spatial-G_n0JvVf-py3.8/bin/python -m pip install --upgrade pip' command.


In [30]:
%load_ext autoreload
%autoreload 2
import h5py
import anndata
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sp
import scipy.sparse.linalg
rng=np.random.default_rng()
import tqdm.notebook
import pickle
%matplotlib inline
import sys
import ipywidgets
import sklearn.neighbors

original_url= "https://datadryad.org/stash/downloads/file_stream/67671"
csv_location='/data/spatial/moffit_merfish/original_file.csv'
h5ad_location='/data/spatial/moffit_merfish/original_file.h5ad'
connectivity_matrix_template='/data/spatial/moffit_merfish/connectivity_%dneighbors.h5ad'
genetypes_location='/data/spatial/moffit_merfish/genetypes.pkl'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# download csv

# munge into hdf5 file

# supplement hdf5 file with a column indicating "tissue id" for each cell

# create global graph (using 3 nearest neigbors)

# write down ligand/receptor sets

# run a simple experiment: use ligands and receptors to predict response genes in excitatory cells, with a linear model

In [31]:
# load data
nneigh=3
ad=anndata.read_h5ad(h5ad_location)
connectivity_matrix=anndata.read_h5ad(connectivity_matrix_template%nneigh).X
gene_lookup={x:i for (i,x) in enumerate(ad.var.index)}

with open(genetypes_location,'rb') as f:
    genetypes=pickle.load(f)

In [32]:
# onehot encode cell classes
def oh_encode(lst):
    lst=np.array(lst)
    group_names=np.unique(lst)
    group_indexes=np.zeros((len(lst),len(group_names)),dtype=bool)
    for i,nm in enumerate(group_names):
        group_indexes[lst==nm,i]=True
    return group_names,group_indexes
cell_classes,cell_class_onehots=oh_encode(ad.obs['Cell_class'])

In [33]:
# a function to construct a prediction problem for a subset of cells

def construct_problem(mask,target_gene,neighbor_genes,self_genes,filter_excitatory=False):
    '''
    mask -- set of cells
    target_gene -- gene to predict
    neighbor_genes -- names of genes which will be read from neighbors
    self_genes -- names of genes which will be read from target cell
    '''
    
    # load subset of data relevant to mask
    local_processed_expression=np.log1p(ad.X[mask].astype(float)) # get expression on subset of cells
    local_edges=connectivity_matrix[mask][:,mask]   # get edges for subset
    
    selfset_idxs=[gene_lookup[x] for x in self_genes] # collect the column indexes associated with them
    selfset_exprs = local_processed_expression[:,selfset_idxs] # collect ligand and receptor expressions
       
    neighborset_idxs=[gene_lookup[x] for x in neighbor_genes] # collect the column indexes associated with them
    neighset_exprs = local_processed_expression[:,neighborset_idxs] # collect ligand and receptor expressions
        
    n_neighs=(local_edges@np.ones(local_edges.shape[0]))
    neigh_avgs = (local_edges@neighset_exprs) / n_neighs[:,None] # average ligand/receptor for neighbors
    
    neigh_cellclass_avgs = (local_edges@cell_class_onehots[mask]) / n_neighs[:,None] # celltype simplex
    
    positions=np.array(ad.obs[['Centroid_X','Centroid_Y','Bregma']])[mask] # get positions
    
    covariates=np.c_[selfset_exprs,neigh_avgs,neigh_cellclass_avgs,positions] # collect all covariates
    predict = local_processed_expression[:,gene_lookup[target_gene]] # collect what we're supposed to predict
    
    if filter_excitatory:
    
        excites=(ad.obs['Cell_class']=='Excitatory')[mask] # get the subset of these cells which are excitatory
        covariates=covariates[excites] # subset to excites
        predict=predict[excites]       # subset to excites
    
    return covariates,predict

In [34]:
neighset=genetypes['ligands']
oset=np.r_[genetypes['ligands'],genetypes['receptors']]
# oset=neighset

# oset=[]
# neighset=[]

trainX,trainY=construct_problem((ad.obs['Animal_ID']>=2)&(ad.obs['Animal_ID']<=4),'Ace2',neighset,oset)
testX,testY=construct_problem((ad.obs['Animal_ID']==1),'Ace2',neighset,oset)

print(trainX.shape,trainY.shape)
print(testX.shape,testY.shape)

# whiten covariates
# mu=np.mean(trainX,axis=0)
# sig=np.std(trainX,axis=0)
# trainX=(trainX-mu)/sig
# testX=(testX-mu)/sig

(131693, 121) (131693,)
(73655, 121) (73655,)


In [7]:
neighset

array(['Cbln1', 'Cxcl14', 'Cbln2', 'Vgf', 'Scg2', 'Cartpt', 'Tac2',
       'Bdnf', 'Bmp7', 'Cyr61', 'Fn1', 'Fst', 'Gad1', 'Ntng1', 'Pnoc',
       'Selplg', 'Sema3c', 'Sema4d', 'Serpine1', 'Adcyap1', 'Cck', 'Crh',
       'Gal', 'Gnrh1', 'Nts', 'Oxt', 'Penk', 'Sst', 'Tac1', 'Trh', 'Ucn3'],
      dtype='<U8')

In [8]:
model=sklearn.linear_model.Ridge(alpha=1.0)
model.fit(trainX,trainY)
np.mean(np.abs(model.predict(testX)-testY))

0.17033753860011874

In [9]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
model=HistGradientBoostingRegressor(loss="squared_error", min_samples_leaf=2, verbose=1, random_state=129, max_iter=10000, n_iter_no_change=100)
model.fit(trainX,trainY)
np.mean(np.abs(model.predict(testX)-testY))

Binning 0.115 GB of training data: 

/home/roko/.cache/pypoetry/virtualenvs/spatial-G_n0JvVf-py3.8/lib/python3.8/site-packages/sklearn/experimental/enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


2.146 s
Binning 0.013 GB of validation data: 0.085 s
Fitting gradient boosted rounds:
[1/10000] 1 tree, 31 leaves, max depth = 13, train loss: 0.04677, val loss: 0.04547, in 0.090s
[2/10000] 1 tree, 31 leaves, max depth = 12, train loss: 0.04527, val loss: 0.04426, in 0.081s
[3/10000] 1 tree, 31 leaves, max depth = 12, train loss: 0.04403, val loss: 0.04334, in 0.088s
[4/10000] 1 tree, 31 leaves, max depth = 11, train loss: 0.04296, val loss: 0.04260, in 0.078s
[5/10000] 1 tree, 31 leaves, max depth = 15, train loss: 0.04208, val loss: 0.04201, in 0.080s
[6/10000] 1 tree, 31 leaves, max depth = 8, train loss: 0.04133, val loss: 0.04153, in 0.070s
[7/10000] 1 tree, 31 leaves, max depth = 11, train loss: 0.04067, val loss: 0.04111, in 0.076s
[8/10000] 1 tree, 31 leaves, max depth = 10, train loss: 0.04012, val loss: 0.04072, in 0.080s
[9/10000] 1 tree, 31 leaves, max depth = 14, train loss: 0.03963, val loss: 0.04041, in 0.075s
[10/10000] 1 tree, 31 leaves, max depth = 12, train loss: 0.

[86/10000] 1 tree, 31 leaves, max depth = 12, train loss: 0.03189, val loss: 0.03877, in 0.050s
[87/10000] 1 tree, 31 leaves, max depth = 11, train loss: 0.03184, val loss: 0.03877, in 0.059s
[88/10000] 1 tree, 31 leaves, max depth = 13, train loss: 0.03179, val loss: 0.03877, in 0.070s
[89/10000] 1 tree, 31 leaves, max depth = 10, train loss: 0.03176, val loss: 0.03878, in 0.074s
[90/10000] 1 tree, 31 leaves, max depth = 12, train loss: 0.03171, val loss: 0.03877, in 0.079s
[91/10000] 1 tree, 31 leaves, max depth = 10, train loss: 0.03167, val loss: 0.03878, in 0.051s
[92/10000] 1 tree, 31 leaves, max depth = 11, train loss: 0.03161, val loss: 0.03877, in 0.063s
[93/10000] 1 tree, 31 leaves, max depth = 10, train loss: 0.03157, val loss: 0.03877, in 0.061s
[94/10000] 1 tree, 31 leaves, max depth = 11, train loss: 0.03153, val loss: 0.03876, in 0.064s
[95/10000] 1 tree, 31 leaves, max depth = 12, train loss: 0.03148, val loss: 0.03877, in 0.064s
[96/10000] 1 tree, 31 leaves, max depth 

0.16080342723552601

### Same 3 cells as above but w/ standardizing this time.

In [10]:
response_genes=['Ace2', 'Aldh1l1', 'Amigo2', 'Ano3', 'Aqp4', 'Ar', 'Arhgap36',
       'Baiap2', 'Ccnd2', 'Cd24a', 'Cdkn1a', 'Cenpe', 'Chat', 'Coch',
       'Col25a1', 'Cplx3', 'Cpne5', 'Creb3l1', 'Cspg5', 'Cyp19a1',
       'Cyp26a1', 'Dgkk', 'Ebf3', 'Egr2', 'Ermn', 'Esr1', 'Etv1',
       'Fbxw13', 'Fezf1', 'Gbx2', 'Gda', 'Gem', 'Gjc3', 'Greb1',
       'Irs4', 'Isl1', 'Klf4', 'Krt90', 'Lmod1', 'Man1a', 'Mbp', 'Mki67',
       'Mlc1', 'Myh11', 'Ndnf', 'Ndrg1', 'Necab1', 'Nnat', 'Nos1',
       'Npas1', 'Nup62cl', 'Omp', 'Onecut2', 'Opalin', 'Pak3', 'Pcdh11x',
       'Pgr', 'Plin3', 'Pou3f2', 'Rgs2', 'Rgs5', 'Rnd3', 'Scgn',
       'Serpinb1b', 'Sgk1', 'Slc15a3', 'Slc17a6', 'Slc17a8', 'Slco1a4',
       'Sln', 'Sox4', 'Sox6', 'Sox8', 'Sp9', 'Synpr', 'Syt2', 'Syt4',
       'Sytl4', 'Th', 'Tiparp', 'Tmem108', 'Traf4', 'Ttn', 'Ttyh2']

import time
import json
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor

all_MAEs = []

time_dict = {}
L1_loss_dict = {}

for animal in [1,2,3,4]:
    start = time.time()
    MAE_list = []
    for target_gene in response_genes:
        neighset=genetypes['ligands']
        oset=np.r_[genetypes['ligands'],genetypes['receptors']]
        # oset=neighset

        # oset=[]
        # neighset=[]
        
        train_animals = [1,2,3,4]
        train_animals.remove(animal)
        print(train_animals)
        # FIX THIS SO THAT ONLY FIRST 4 ANIMALS GET USED
        trainX,trainY=construct_problem((ad.obs['Animal_ID']!=animal)&(ad.obs['Animal_ID']<=4),target_gene,neighset,oset)
        testX,testY=construct_problem((ad.obs['Animal_ID']==animal),target_gene,neighset,oset)

        print(trainX.shape,trainY.shape)
        print(testX.shape,testY.shape)

        # whiten covariates
        mu=np.mean(trainX,axis=0)
        sig=np.std(trainX,axis=0)
        trainX=(trainX-mu)/sig
        testX=(testX-mu)/sig

        model=HistGradientBoostingRegressor(loss="absolute_error")
        model.fit(trainX,trainY)
        MAE_list.append(np.mean(np.abs(model.predict(testX)-testY)))

    end = time.time()
    time_dict[f"Female_Naive_{animal}"] = end-start
    L1_loss_dict[f"Female_Naive_{animal}"] = float(np.mean(MAE_list))

    with open("XGBoost_L1_time.json", "w") as outfile:
        json.dump(time_dict, outfile, indent=4)

    with open("XGBoost_L1_MAE.json", "w") as outfile:
        json.dump(L1_loss_dict, outfile, indent=4)
    
    all_MAEs.append(np.mean(MAE_list))
    
print(np.mean(all_MAEs))

[2, 3, 4]
(131693, 121) (131693,)
(73655, 121) (73655,)
[2, 3, 4]
(131693, 121) (131693,)
(73655, 121) (73655,)
[2, 3, 4]
(131693, 121) (131693,)
(73655, 121) (73655,)
[2, 3, 4]
(131693, 121) (131693,)
(73655, 121) (73655,)
[2, 3, 4]
(131693, 121) (131693,)
(73655, 121) (73655,)
[2, 3, 4]
(131693, 121) (131693,)
(73655, 121) (73655,)
[2, 3, 4]
(131693, 121) (131693,)
(73655, 121) (73655,)
[2, 3, 4]
(131693, 121) (131693,)
(73655, 121) (73655,)
[2, 3, 4]
(131693, 121) (131693,)
(73655, 121) (73655,)
[2, 3, 4]
(131693, 121) (131693,)
(73655, 121) (73655,)
[2, 3, 4]
(131693, 121) (131693,)
(73655, 121) (73655,)
[2, 3, 4]
(131693, 121) (131693,)
(73655, 121) (73655,)
[2, 3, 4]
(131693, 121) (131693,)
(73655, 121) (73655,)
[2, 3, 4]
(131693, 121) (131693,)
(73655, 121) (73655,)
[2, 3, 4]
(131693, 121) (131693,)
(73655, 121) (73655,)
[2, 3, 4]
(131693, 121) (131693,)
(73655, 121) (73655,)
[2, 3, 4]
(131693, 121) (131693,)
(73655, 121) (73655,)
[2, 3, 4]
(131693, 121) (131693,)
(73655, 121) (

[1, 3, 4]
(136309, 121) (136309,)
(69039, 121) (69039,)
[1, 3, 4]
(136309, 121) (136309,)
(69039, 121) (69039,)
[1, 3, 4]
(136309, 121) (136309,)
(69039, 121) (69039,)


KeyboardInterrupt: 

In [ ]:
response_genes=['Ace2', 'Aldh1l1', 'Amigo2', 'Ano3', 'Aqp4', 'Ar', 'Arhgap36',
       'Baiap2', 'Ccnd2', 'Cd24a', 'Cdkn1a', 'Cenpe', 'Chat', 'Coch',
       'Col25a1', 'Cplx3', 'Cpne5', 'Creb3l1', 'Cspg5', 'Cyp19a1',
       'Cyp26a1', 'Dgkk', 'Ebf3', 'Egr2', 'Ermn', 'Esr1', 'Etv1',
       'Fbxw13', 'Fezf1', 'Gbx2', 'Gda', 'Gem', 'Gjc3', 'Greb1',
       'Irs4', 'Isl1', 'Klf4', 'Krt90', 'Lmod1', 'Man1a', 'Mbp', 'Mki67',
       'Mlc1', 'Myh11', 'Ndnf', 'Ndrg1', 'Necab1', 'Nnat', 'Nos1',
       'Npas1', 'Nup62cl', 'Omp', 'Onecut2', 'Opalin', 'Pak3', 'Pcdh11x',
       'Pgr', 'Plin3', 'Pou3f2', 'Rgs2', 'Rgs5', 'Rnd3', 'Scgn',
       'Serpinb1b', 'Sgk1', 'Slc15a3', 'Slc17a6', 'Slc17a8', 'Slco1a4',
       'Sln', 'Sox4', 'Sox6', 'Sox8', 'Sp9', 'Synpr', 'Syt2', 'Syt4',
       'Sytl4', 'Th', 'Tiparp', 'Tmem108', 'Traf4', 'Ttn', 'Ttyh2']

import time
import json
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor

all_MAEs = []

time_dict = {}
L1_loss_dict = {}

for animal in [1,2,3,4]:
    start = time.time()
    MAE_list = []
    for target_gene in response_genes:
        neighset=genetypes['ligands']
        oset=np.r_[genetypes['ligands'],genetypes['receptors']]
        # oset=neighset

        # oset=[]
        # neighset=[]
        
        train_animals = [1,2,3,4]
        train_animals.remove(animal)
        print(train_animals)
        # FIX THIS SO THAT ONLY FIRST 4 ANIMALS GET USED
        trainX,trainY=construct_problem((ad.obs['Animal_ID']!=animal)&(ad.obs['Animal_ID']<=4),target_gene,neighset,oset,True)
        testX,testY=construct_problem((ad.obs['Animal_ID']==animal),target_gene,neighset,oset,True)

        print(trainX.shape,trainY.shape)
        print(testX.shape,testY.shape)

        # whiten covariates
        mu=np.mean(trainX,axis=0)
        sig=np.std(trainX,axis=0)
        trainX=(trainX-mu)/sig
        testX=(testX-mu)/sig

        model=HistGradientBoostingRegressor(loss="absolute_error")
        model.fit(trainX,trainY)
        MAE_list.append(np.mean(np.abs(model.predict(testX)-testY)))

    end = time.time()
    time_dict[f"Female_Naive_{animal}"] = end-start
    L1_loss_dict[f"Female_Naive_{animal}"] = float(np.mean(MAE_list))

    with open("XGBoost_L1_time_excitatory.json", "w") as outfile:
        json.dump(time_dict, outfile, indent=4)

    with open("XGBoost_L1_MAE_excitatory.json", "w") as outfile:
        json.dump(L1_loss_dict, outfile, indent=4)
    
    all_MAEs.append(np.mean(MAE_list))
    
print(np.mean(all_MAEs))

In [41]:
neighset=genetypes['ligands']
oset=np.r_[genetypes['ligands'],genetypes['receptors']]
# oset=neighset

# oset=[]
# neighset=[]

trainX,trainY=construct_problem((ad.obs['Animal_ID']<=30),'Th',neighset,oset)
testX,testY=construct_problem((ad.obs['Animal_ID']>30),'Th',neighset,oset)

print(trainX.shape,trainY.shape)
print(testX.shape,testY.shape)

# whiten covariates
mu=np.mean(trainX,axis=0)
sig=np.std(trainX,axis=0)
trainX=(trainX-mu)/sig
testX=(testX-mu)/sig

(893456, 121) (893456,)
(134392, 121) (134392,)


In [42]:
model=sklearn.linear_model.Ridge(alpha=1.0)
model.fit(trainX,trainY)
np.mean(np.abs(model.predict(testX)-testY))

0.06757797610786355

In [43]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
model=HistGradientBoostingRegressor(loss="absolute_error")
model.fit(trainX,trainY)
np.mean(np.abs(model.predict(testX)-testY))

0.05254223324894638

### Comparison to Standard Scalar

In [44]:
neighset=genetypes['ligands']
oset=np.r_[genetypes['ligands'],genetypes['receptors']]
# oset=neighset

# oset=[]
# neighset=[]

trainX,trainY=construct_problem((ad.obs['Animal_ID']<=30),'Th',neighset,oset)
testX,testY=construct_problem((ad.obs['Animal_ID']>30),'Th',neighset,oset)

mu=np.mean(trainX,axis=0)
sig=np.std(trainX,axis=0)
trainX_Jackson=(trainX-mu)/sig
testX_Jackson=(testX-mu)/sig

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(trainX)
trainX_Roman = scaler.transform(trainX)

In [45]:
model=sklearn.linear_model.Ridge(alpha=1.0)
model.fit(trainX,trainY)
np.mean(np.abs(model.predict(testX)-testY))

0.06757765995432916

In [46]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
model=HistGradientBoostingRegressor(loss="absolute_error")
model.fit(trainX,trainY)
np.mean(np.abs(model.predict(testX)-testY))

0.05254223324894638

In [49]:
sum(model.predict(testX))

0.0